# py2neo使用

## 创建节点和关系

In [1]:
from py2neo import Node, Relationship,Graph

In [2]:
graph = Graph('bolt://localhost:7687',username='neo4j', password='123456')

In [3]:
alice = Node('Person', name='Alice')
bob = Node('Person', name='Bob')
relationship = Relationship(alice, 'KNOWS', bob)
print(alice, bob, relationship)

(:Person {name: 'Alice'}) (:Person {name: 'Bob'}) (Alice)-[:KNOWS {}]->(Bob)


### 添加属性

Node和Relationship都继承了PropertyDict类，它可以赋值很多属性，类似于字典的形式。

In [4]:
alice['age'] = 20
bob['age'] = 21
relationship['time'] = '2020/02/02'
print(alice, bob, relationship)

(:Person {age: 20, name: 'Alice'}) (:Person {age: 21, name: 'Bob'}) (Alice)-[:KNOWS {time: '2020/02/02'}]->(Bob)


#### 添加默认属性

也可以通过类似字典的方式，使用setdefault添加默认属性

In [5]:
alice.setdefault('location', '北京')
print(alice)

(:Person {age: 20, location: '\u5317\u4eac', name: 'Alice'})


可见没有给alice对象赋值location属性，它就会使用默认属性。

In [6]:
alice['location'] = '上海'
alice.setdefault('location', '北京')
print(alice)

(:Person {age: 20, location: '\u4e0a\u6d77', name: 'Alice'})


但是如果赋值了location属性，那么它就会覆盖默认属性

### 批量更新属性

In [8]:
data = {
    'name':'Amy',
    'age':21
}

alice.update(data)
print(alice)

(:Person {age: 21, location: '\u4e0a\u6d77', name: 'Amy'})


旧有的属性会被保留，新旧都有的会被更新

## Subgraph 子图

子图是Node和Relationship的集合，最简单的构造子图的方式是通过关系运算符

In [9]:
from py2neo import Node, Relationship

In [11]:
alice = Node('Person', name='Alice')
bob = Node('Person', name='Bob')
relationship = Relationship(alice, 'KNOWS', bob)
sub_graph = alice | bob | relationship
print(sub_graph)

Subgraph({Node('Person', name='Alice'), Node('Person', name='Bob')}, {KNOWS(Node('Person', name='Alice'), Node('Person', name='Bob'))})


### 添加数据

其实上面的一系列操作，都没有实际保存到neo4j数据库中

In [13]:
from py2neo import Node,Graph

In [15]:
graph = Graph('bolt://localhost:7687', username='neo4j', password='123456')
transaction = graph.begin()

In [16]:
alice = Node('Person', name='Alice', location='北京', age=21)
bob = Node('Person', name='Bob', location='上海', age=22)
mike = Node('Person', name='Mike', location='重庆', age=21 )

In [17]:
transaction.create(alice | bob | mike)
transaction.commit()

<Bookmark 'FB:kcwQpR792zMVRG6xUlW8tVwKOWiQ'>

![](https://i.loli.net/2021/01/23/z8O7h1xk3KZ9VIJ.png)

### 查询数据

使用node和relationship数据类型自带的匹配属性

In [18]:
from py2neo import Graph

In [19]:
graph = Graph('bolt://localhost:7687', username='neo4j', password='123456')

In [22]:
# TODO 据说是效率较慢，没测试过
result = graph.nodes.match('Person', age=21)
print(list(result))

[Node('Person', age=21, location='北京', name='Alice'), Node('Person', age=21, location='重庆', name='Mike')]


使用查询模块

In [23]:
from py2neo import Graph
from py2neo.matching import NodeMatcher

In [25]:
graph = Graph('bolt://localhost:7687', username='neo4j', password='123456')
node_matcher = NodeMatcher(graph=graph)

In [26]:
# 写法一
result = node_matcher.match('Person', age=21)
print(list(result))

[Node('Person', age=21, location='北京', name='Alice'), Node('Person', age=21, location='重庆', name='Mike')]


In [31]:
# 写法二
result = node_matcher.match('Person').where(age=21)
print(list(result))

[Node('Person', age=21, location='北京', name='Alice'), Node('Person', age=21, location='重庆', name='Mike')]


上面的几个操作就相当于：

```cypher
match (p:Person) where p.age=21 return p
```

In [29]:
# 取第一个
result = node_matcher.match('Person', age=21).first()
print(result)

(_10:Person {age: 21, location: '\u5317\u4eac', name: 'Alice'})


```cypher
match (p:Person) where p.age=21 return p limit 1
```

以上的py2neo匹配，其缺点在于匹配被人为的分割成了节点和关系的匹配，使用起来不够灵活，因为节点和关系的匹配常常是紧密关联的。不如原生的cypher灵活，它没有限定类型